<a href="https://colab.research.google.com/github/uio-fys-mek/md-prosjekt/blob/master/vektorisering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

# Vektorisering

Når vi snakker om *vektorisering* i Python mener vi egentlig å utnytte funksjonaliteter og vektoroperasjoner i Python-pakken `numpy`. Numpy er en pakke som er skrevet i C/C++ og delvis Fortran - lavnivåspråk som generelt er mye raskere enn høynivåspråk som f.eks. Python. Dette gjør seg virkelig gjeldende i essensielle funksjonaliteter som for- og while-løkker. For eksempel vil en for-løkke i C/C++ gå 100 ganger (!) raskere enn en for-løkke i Python.

## Enkelt eksempel
Det som er fint med numpy-biblioteket er at vektoroperasjoner som gjøres på numpy-arrays vil kalle på funksjonaliteter skrevet i nettopp disse lavnivå språkene. På denne måten drar man fordelene av hastighetene til disse, samtidig som man beholder en ren og kort python-syntax. Dette lar oss tilsynelatende gjøre operasjoner på *hele arrays* om gangen i python-scriptene våre.

Enkelt eksempel:


In [0]:
a = np.array([1,2,3])
b = np.array([1,1,1])
c = a - b
c

array([0, 1, 2])

Dette oversettes til C/Fortran-kode som tilsvarer

In [0]:
c = np.zeros_like(a)
for i in range(len(c)):
    c[i] = a[i] - b[i]
c

array([0, 1, 2])

Dette går mye raskere:

In [0]:
def dum():
    N = int(1E6)
    a = np.zeros(N)
    b = np.zeros(N)
    c = np.zeros(N)
    for i in range(N):
        a[i] = np.random.uniform()
        b[i] = np.random.uniform()
        c[i] = a[i] - b[i]

def vektorisert():
    N = int(1E6)
    a = np.random.uniform(size=N)
    b = np.random.uniform(size=N)
    c = a - b

In [0]:
%timeit dum()

1 loop, best of 3: 2.08 s per loop


In [0]:
%timeit vektorisert()

10 loops, best of 3: 32.3 ms per loop


Som vi ser, går den vektoriserte koden mye raskere. I tillegg er koden enklere og mer matematikknær.

Moralen er at man alltid bør prøve å erstatte så mange Python-løkker som mulig med vektoriserte numpy-operasjoner.

## Parvise operasjoner
I Grand Challenge må vi finne avstandene mellom hvert eneste atompar for å kunne regne ut summen av kreftene på hvert atom. I eksemplene under viser vi flere måter å sammenligne alle elementene i et array mot hverandre, både delvis og fullt vektorisert. For enkelhets skyld opererer vi på et 2D-array, men metodene kan også lett brukes på arrays med flere dimensjoner.


### Enkel tilnærming
Hvis posisjonene er lagret i `r` som har dimensjon $N \times 3 $, hvor $N$ er antallet partikler, vil koden typisk se slik ut:
```python
for i in range(N-1):
    for j in range(i+1, N):
        dr = r[j] - r[i]
```

Dette er den mest "rett frem"-måten å sammenligne array'ene på, men også den tregeste. Når antallet iterasjoner per loop ($N$) blir stort, vil bare det å loope gjennom løkkene ta en god del tid. Skal man i tillegg gjøre kompliserte beregninger inne i løkkene kan det være en fordel å ha abonnement hos Netflix. Samtidig er det veldig oversiktlig og enkelt å kun behandle ett element om gangen, hvor man har god kontroll på hva som faktisk skjer. Likevel, 


 - +Lett å forstå hva som skjer.
 - +Trenger kun ett element i minnet om gangen.
 - -Kan bli en del kode.
 - -*Ekstremt* tregt.

### Vektorisert utregning
Alle avstandsvektorene til naboene til atom $i$ kan regnes ut på én gang med
```python
for i in range(N):
    drs = r - r[i]
```
og kreftene kan regnes ut fra disse. OBS: `drs[i]` vil være $\vec{0}$. Vil man unngå disse $\vec{0}$-vektorene kan enten fjerne dette elementet i etterkant, eller indeksere direkte med " `r[np.arange(N)!=i] - r[i]` ".

Ved å vektorisere på denne måten kan man forsatt ta i bruk "Newtons 3. lov" (avstanden $r_{ij} = - r_{ji}$), men da trenger man også et ($N \times N \times 3$)-array utenfor hvor man kan sette inn de aktuelle avstandene.

```python
for i in range(N):
    drs = r[i+1:] - r[i]
    dr_matr[i, i+1:] = drs
    dr_matr[i+1: i] = -drs
```


- +Fjerner én for-løkke.
- +Forholdsvis rett frem for flerdimensjonale arrays.
- -Fortsatt én for-løkke.
- -Kryptisk indeksering.

### Ekstremvektorisering
*Broadcasting* kan brukes til å regne ut avstandene mellom atomene helt uten for-løkker. Se på dette eksemplet:

In [0]:
x1 = np.array([1,2,3])
x2 = x1.reshape(-1,1)
print(x1, "\n", x2)

[1 2 3] 
 [[1]
 [2]
 [3]]


In [0]:
x1 - x2

array([[ 0,  1,  2],
       [-1,  0,  1],
       [-2, -1,  0]])

Vi ser at når en kolonevektor trekkes fra en radvektor, blir resultatet en matrise hvor første rad er radvektoren minus første element i kolonnevektoren etc. Dette kan brukes til å regne ut alle innbyrdes avstande, uten en eneste for-løkke.

- +Ingen for-løkker.
- +Raskest.
- -Ingen Newtons 3. lov.
- -Krever $N^2$ elementer i minnet - begrenser systemstørrelse.
- Grisete, lite intuitiv kode for flerdimensjonale arrays.

## Andre alternativer
### Numba
Numba er en Python-pakke som oversetter Python-kode til LLVM-kode og kompilerer den. Resultatet er at Python-koden kan gå nesten like raskt som de kompilerte språkene.

Eksempel:

In [0]:
import numba

@numba.njit
def K_numba(v):
    K = 0
    N = len(v)
    for i in range(N):
        for j in range(3):
            K += v[i,j]**2
    return 0.5*K

def K_dum(v):
    K = 0
    N = len(v)
    for i in range(N):
        for j in range(3):
            K += v[i,j]**2
    return 0.5*K

def K_numpy(v):
    return 0.5*np.sum(v**2)

In [0]:
N = int(1E7)
v = np.random.normal(size=(N,3))

In [0]:
%timeit K_numba(v)

10 loops, best of 3: 46.8 ms per loop


In [0]:
%timeit K_dum(v)

1 loop, best of 3: 16.5 s per loop


In [0]:
%timeit K_numpy(v)

10 loops, best of 3: 97.5 ms per loop


Her ser vi at én numpy-operasjon kan være raskere enn numba. Her var vi dog heldige med at numpy har en funksjon som gjør akkurat det vi vil. I de fleste tilfeller vil numba være raskere, selv med mer intuitiv kode.

Ved bruk av numba er det viktig å bare bruke tall og numpy-arrayer.  Ved å bruke `njit` (i stedet for `jit`) sjekker numba at dette er oppfylt.

### Fortran
Det er enkelt å kalle på Fortran-kode som gjør den tyngste delen av utregningen.

Gitt følgende fil:

In [0]:
%%file k_fortran.f90
function K_fortran(v, N) result(K)
    ! Argumenter
    integer, intent(in) :: N
    double precision, intent(in) :: v(N, 3)
        
    ! Returverdi
    double precision :: K
    
    K = 0.5*sum(v**2)
end function

Overwriting k_fortran.f90


Denne kan kompileres til en python-modul med terminalkommandoen

In [0]:
! f2py3 -c -m fortran_modul k_fortran.f90 > /dev/null

Modulen kan så importeres og kjøres:

In [0]:
from fortran_modul import k_fortran

v = np.asarray(v, order="F") # Fortran-minne-ordning

In [0]:
%timeit k_fortran(v, N)

10 loops, best of 3: 53.1 ms per loop
